In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from llserver.utils.handler import UniserverHandler

In [ ]:
handler = UniserverHandler(port=8000)

In [ ]:
handler.get_running_models()

In [ ]:
handler.start_model("lera_api")

In [ ]:
info = handler.get_running_models()
model_id = list(info["running_models"]["models"].keys())[0]
model_id

In [ ]:
prompt = "compare images, describe the difference between them"
base_path = "/llserver/data/"
image_paths = [
    base_path + "lera_test_replanner/0.png",
    base_path + "metatmp.png",
]
# model = "gemini-pro-1.5"
model = "gpt-4o"

In [ ]:
put_response = handler.put_task(model_id=model_id, prompt=prompt, image_paths=image_paths, model=model)
task_id = put_response["task_id"]["task_id"]
task_id

In [ ]:
status = ""
wait_time = 3
while status != "completed":
    time.sleep(wait_time)
    result_response = handler.get_task_result(model_id=model_id, task_id=task_id)
    status = result_response.get("status")
    print(f"Status: {status}")

result = result_response.get("result")
print(result)

In [ ]:
handler.stop_model(model_id=model_id)

In [ ]:
handler.stop_all_models()

# Test Lera API

In [1]:
# LLServer and Relanner

import os
from llserver.utils.handler import UniserverHandler
import json
import time
from PIL import Image
import numpy as np

def save_images(images, dirs):
    for idx, (image, dir) in enumerate(zip(images, dirs)):
        Image.fromarray(image).save(dir)
    return dirs

In [2]:
# Input

instruction = "Move plates, wineglasses, mugs, dishbowls from the table into the dishwasher and then turn it on."
suc_actions = [
    f'[Walk] <kitchentable> (226)',
    f'[Walk] <bench> (227)',
#    f'[Walk] <bananas> (1001)',
#    f'[Grab] <bananas> (1001)',
]
next_actions = [
#    f'[Walk] <kitchentable> (226)',
#    f'[Walk] <bench> (227)',
    f'[Walk] <bananas> (1001)',
    f'[Grab] <bananas> (1001)',
]
# image_path = r"C:\Users\prestige\Desktop\Work\virtualhome100\windows_sim\Output\script\Action_0082_normal.png"
image_path = "/home/mpatratskiy/work/meta_world/llserver/data/metatmp.png"
images = [np.array(Image.open(image_path))]

In [9]:
# Replanning initialization

handler = UniserverHandler(port=8000)
start_response = handler.start_model("lera_api")
model_id = start_response["model_id"]

experiment_name = "VhomeDebug"
model_name = "gpt-4o" #+ "###" + "alfred"
# save_base_dir = f'C:\\Users\\prestige\\Desktop\\Work\\virtualhome100\\llserver\\data\\{experiment_name}'
# load_base_dir = f"/llserver/data/{experiment_name}"

save_base_dir = f"/home/mpatratskiy/work/meta_world/llserver/data/{experiment_name}"
load_base_dir = f"/llserver/data/{experiment_name}"
os.makedirs(save_base_dir, exist_ok=True)

Server is ready to go


In [10]:
# Process input

prompt = f"""{instruction}###{suc_actions}###{next_actions}###"""

dirs_to_save = [f"{save_base_dir}/{idx}.png" for idx in range(len(images))]
dirs_to_load = [f"{load_base_dir}/{idx}.png" for idx in range(len(images))]
save_images(images, dirs_to_save)

['/home/mpatratskiy/work/meta_world/llserver/data/VhomeDebug/0.png']

In [11]:
put_response = handler.put_task(model_id=model_id, prompt=prompt, image_paths=dirs_to_load, model=model_name)
task_id = put_response["task_id"]["task_id"]

{'model': 'gpt-4o'}
{'model_id': 'e5bb4593-5683-49b2-8f78-f5800adfdbe3', 'prompt': "Move plates, wineglasses, mugs, dishbowls from the table into the dishwasher and then turn it on.###['[Walk] <kitchentable> (226)', '[Walk] <bench> (227)']###['[Walk] <bananas> (1001)', '[Grab] <bananas> (1001)']###", 'image_paths': ['/llserver/data/VhomeDebug/0.png'], 'extra_params': {'model': 'gpt-4o'}}


In [12]:
status = ""
wait_time = 13
while status != "completed":
    time.sleep(wait_time)
    result_response = handler.get_task_result(model_id=model_id, task_id=task_id)
    status = result_response.get("status")
    print(f"Status: {status}")

if status == "not found":
    raise Exception("Task failed: Task not found")

info = result_response.get("result")
print(info)

Status: completed
{'[goal]': 'Move plates, wineglasses, mugs, dishbowls from the table into the dishwasher and then turn it on.', '[success_actions]': "['[Walk] <kitchentable> (226)', '[Walk] <bench> (227)']", '[current_plan]': "['[Walk] <bananas> (1001)', '[Grab] <bananas> (1001)']", '[available_actions]': '', '[look_response]': "The scene shows a robot arm positioned over a table. Here's a detailed description:\n\n- **Robot Arm**: The robot arm is red and appears to be mid-action over the table. It is extended, with its gripper hovering just above the table surface.\n  \n- **Gripper**: The gripper has its pincers open, positioned to either grab or release an object. It is currently empty, indicating that it may have accidentally dropped an object during movement.\n\n- **Objects on the Table**: \n  - A red cylinder lies directly under the gripper, suggesting it might have been dropped.\n  - A blue sphere is located slightly to the right of the gripper.\n  - The rest of the table appea

In [14]:
print(info.get("[replan_response]"))

["locate('plate')", "pick('plate')", "locate('dishwasher')", "place_on_top_of('dishwasher')", "locate('wineglass')", "pick('wineglass')", "locate('dishwasher')", "place_on_top_of('dishwasher')", "locate('mug')", "pick('mug')", "locate('dishwasher')", "place_on_top_of('dishwasher')", "locate('dishbowl')", "pick('dishbowl')", "locate('dishwasher')", "place_on_top_of('dishwasher')", "done()"]
